# Составление фондового индекса

## Вы научитесь:

• работать с методом главных компонент

• использовать его для вычисления улучшенного индекса Доу-Джонса

## Введение

Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.
Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

## Данные
В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов).

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.

## Реализация в Sklearn

Загрузим данные close_prices.csv. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода:

In [20]:
import pandas as pd

data_train=pd.read_csv('close_prices.csv')

На загруженных данных обучим преобразование PCA с числом компоненты равным 10:

In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
result=pca.fit(data_train.iloc[:,1:31])

variance_ratio=result.explained_variance_ratio_
components=result.components_

n = 1
while sum(variance_ratio[:n])<0.9:
    n+=1

print('Достаточно',n,'компонент, чтобы объяснить 90% дисперсии')

Достаточно 4 компонент, чтобы объяснить 90% дисперсии


Применим построенное преобразование к исходным данным и возьмём значения первой компоненты:

In [22]:
result=pca.transform(data_train.iloc[:,1:31])
firstComponent=result[:,0]

Загрузим информацию об индексе Доу-Джонса из файла djia_index.csv:

In [23]:
data_test=pd.read_csv('djia_index.csv')
data_test_y=data_test.iloc[:,1]

Вычислим корреляцию Пирсона между первой компонентой и индексам Доу-Джонса:

In [24]:
from numpy import corrcoef
pearson=corrcoef(firstComponent,data_test_y)
print(round(pearson[0,1],2))

0.91


Вычислим, какая компания имеет наибольший вес в первой компоненте:

In [25]:
firstComponent=pd.DataFrame(data=components[0])
brandIndex=firstComponent.idxmax()[0]+1
print(data_train.columns[brandIndex])

V
